In [1]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import math

#from collections import OrderedDict
#import sklean

In [2]:
# Data params
textures = 11
trials = 100

# Set bin and sim params
#bin_size = 10   # Bin size in ms
bin_sizes = [10, 100, 200, 250, 300] # Bin sizes for different datasets
sim_length = 5000   # Sim time in ms
#bins = math.ceil(sim_length / bin_size)   # Round number of bins up to the nearest integer

In [3]:
# Function to remove repeated spikes from data
# def remove_duplicates(data):
#     for i in range(len(data)):
#         for j in range(len(data[i])): 
#             data[i][j] = list(OrderedDict.fromkeys(data[i][j]))
#     return data

# New numpy based function
def remove_duplicates(data):
    new_data = []
    
    for i in range(len(data)):
        if data[i]:
            new_data.append(np.unique(data[i]))
        
    return new_data

In [4]:
# Function to create an array of labels for the dataset
# Returns a 1d list of texture labels
# NOTE: This function only works when dataset is creted as below
def create_labels(no_textures, no_trials):
    labels = []
    
    for q in range(no_textures):
        for w in range(trials):
            labels.append(q)
            
    return labels

In [5]:
# Import and bin data


# Loop for all bin_sizes
for x in bin_sizes:
    
    print("Bin Size = " + str(x))    
    bins = math.ceil(sim_length / x)   # Round number of bins up to the nearest integer
    dataset = np.empty([int(bins)])

    for kk in range(textures):
        for ll in range(trials):
            # Path to file containing spike timings
            FILE_NAME = "Artificial Dataset " + str(ll) + "Texture No. " + str(kk) + ".pickle"
            DATA_PATH = "/home/farscope2/Documents/PhD/Spiking Nets Project/SpikingNetsTexture/datasets/TacTip_NM/Reduced_natural/" + FILE_NAME

            # Import and flatten the dataset for use in the network
            spike_times = np.load(DATA_PATH, allow_pickle=True)
            spks = spike_times.reshape(-1)

            # Remove duplicates using previous function
            spks = remove_duplicates(spks)

            # List to hold all data from item
            data = []
            value = 0

            # Loop untill all bins are full
            for bin_ms in range(0, sim_length, x):
                # Loop entire population
                for t in range(len(spks)):
                    if spks[t].any():
                        # Check number of times each neuron spiked within this timeframe
                        ar = np.array(spks[t])
                        value += np.count_nonzero(((ar >= bin_ms) & (ar < bin_ms + x)))

                data.append(value)
                value = 0

            # DEBUG
            #print("Trial: " + str(ll) + "Texture: " + str(kk))
            #print("Bin Size =" +str(x))
            #print(dataset)
            #print(data)

            # If dataset is currently empty then dataset is formed from data list
            # Else create row below current dataset to input new data
            if dataset.size == 0:
                dataset[0] = np.array(data)
            else:
                dataset = np.vstack([dataset, data])


            data_dir = "/home/farscope2/Documents/PhD/Spiking Nets Project/SpikingNetsTexture/datasets/TacTip_NM/natural_classifier"
            pickle_out = open(os.path.join(data_dir, str(sim_length) + "ms - " + str(x) + "ms bin size dataset.pickle"), 'wb')
            pickle.dump(dataset, pickle_out)
            pickle_out.close()

Bin Size = 10
Bin Size = 100
Bin Size = 200
Bin Size = 250
Bin Size = 300


In [6]:
# Create label list for dataset
labels = create_labels(textures, trials)

In [7]:
# Pickle the dataset for ease of future use
# This part of the notebook need only be used once

# data_dir = "/home/farscope2/Documents/PhD/Spiking Nets Project/SpikingNetsTexture/datasets/TacTip_NM/natural_classifier"
# pickle_out = open(os.path.join(data_dir, str(sim_length) + "ms - " + str(bin_size) + "ms bin size dataset.pickle"), 'wb')
# pickle.dump(dataset, pickle_out)
# pickle_out.close()

pickle_out = open(os.path.join(data_dir, str(textures) + " textures - " + str(trials) + " trials labels.pickle"), 'wb')
pickle.dump(labels, pickle_out)
pickle_out.close()